In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import pandas as pd
import numpy as np
from scipy import stats
from datetime import timedelta
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [3]:
data = pd.read_csv('../input/ecommerce-data/data.csv', encoding = "ISO-8859-1")

In [4]:
data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'])

In [5]:
print('{:,} rows; {:,} columns'.format(data.shape[0], data.shape[1]))
print('{:,} transactions don\'t have a customer id'.format(data[data.CustomerID.isnull()].shape[0]))
print('Transactions timeframe from {} to {}'.format(data['InvoiceDate'].min(),data['InvoiceDate'].max()))

In [6]:
data.dropna()

In [7]:
data['TotalSum'] = data['Quantity'] * data['UnitPrice']

In [13]:
snapshot_date = data['InvoiceDate'].max() + timedelta(days=1)
print(snapshot_date)

In [18]:
data_preprocess = data.groupby(['CustomerID']).agg({'InvoiceDate': lambda x: (snapshot_date - x.max()).days,'InvoiceNo': 'count','TotalSum': 'sum'})

In [24]:
data_preprocess.rename(columns={'InvoiceDate': 'Recency','InvoiceNo': 'Frequency','TotalSum': 'MonetaryValue'}, inplace=True)

In [26]:
r_labels = range(4, 0, -1)
f_labels = range(1, 5)
m_labels = range(1, 5)
r_groups = pd.qcut(data_preprocess['Recency'], q=4, labels=r_labels)
f_groups = pd.qcut(data_preprocess['Frequency'], q=4, labels=f_labels)
m_groups = pd.qcut(data_preprocess['MonetaryValue'], q=4, labels=m_labels)
data_preprocess = data_preprocess.assign(R = r_groups.values, F = f_groups.values)
data_preprocess = data_preprocess.assign(M = m_groups.values)
data_preprocess.head()

In [31]:
def join_rfm(x): return str(int(x['R'])) + str(int(x['F'])) + str(int(x['M']))
data_preprocess['RFM_Segment_Concat'] = data_preprocess.apply(join_rfm, axis=1)
data_preprocess.head()

In [32]:
data_preprocess['RFM_Score'] = data_preprocess[['R','F','M']].sum(axis=1)

In [34]:
def RFM_Lvl(df):
    if df['RFM_Score'] >= 9:
        return 'Can\'t Loose Them'
    elif ((df['RFM_Score'] >= 8) and (df['RFM_Score'] < 9)):
        return 'Champions'
    elif ((df['RFM_Score'] >= 7) and (df['RFM_Score'] < 8)):
        return 'Loyal/Commited'
    elif ((df['RFM_Score'] >= 6) and (df['RFM_Score'] < 7)):
        return 'Potential'
    elif ((df['RFM_Score'] >= 5) and (df['RFM_Score'] < 6)):
        return 'Promising'
    elif ((df['RFM_Score'] >= 4) and (df['RFM_Score'] < 5)):
        return 'Requires Attention'
    else:
        return 'Demands Activation'
# Create a new variable RFM_Level
data_preprocess['RFM_Level'] = data_preprocess.apply(RFM_Lvl, axis=1)

In [35]:
data_preprocess.head()